## Day 4: Giant Squid

In [16]:
from typing import List
from pathlib import Path
import re


In [3]:
with Path('./4-test-input.txt') as file:
    bingo_text = file.read_text()
    for line in bingo_text.splitlines():
        print(line)
# bingo_text.splitlines()

7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7


In [76]:
class Board:
    def __init__(self, lines: List[str]) -> None:
        self.board: List[List[int]] = [
            [(int(n), False) for n in re.findall('\d+', line)] for line in lines]

    def __repr__(self) -> str:
        return f"Board: {self.board}"

    @property
    def winner(self):
        for row in self.board:
            if sum(n[1] for n in row) >= 5:
                return True
        for col in tuple(zip(*self.board)):
            if sum(n[1] for n in col) >= 5:
                return True
        return False

    @property
    def sum_unmarked(self):
        return sum([value[0] for row in self.board for value in row if not value[1]])

    def mark(self, number):
        for y, row in enumerate(self.board):
            for x, value in enumerate(row):
                if value[0] == number:
                    self.board[y][x] = (number, True)
                    return self.winner
        return self.winner


b = Board("""14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7""".splitlines())

In [78]:
b.sum_unmarked

325

In [79]:
b.mark(2)
b.mark(0)
b.mark(12)
b.mark(3)
# b.mark(7)
print(b.winner)
print(b.sum_unmarked)
b

False
308


Board: [[(14, False), (21, False), (17, False), (24, False), (4, False)], [(10, False), (16, False), (15, False), (9, False), (19, False)], [(18, False), (8, False), (23, False), (26, False), (20, False)], [(22, False), (11, False), (13, False), (6, False), (5, False)], [(2, True), (0, True), (12, True), (3, True), (7, False)]]

In [98]:
class Bingo:
    def __init__(self, input: str) -> None:
        self.lines = input.splitlines()
        self.numbers: List[int] = [int(i) for i in self.lines[0].split(',')]
        self.boards = [Board(self.lines[n*6+2:n*6+7])
                       for n in range(int(len(self.lines)/6))]
        self.winners = []

    @classmethod
    def from_file(cls, filename: str):
        with Path(filename) as file:
            bingo_text = file.read_text()
        return cls(bingo_text)
        
    def draw(self):
        for n in self.numbers:
            winner_index = [i for i, board in enumerate(self.boards) if board.mark(n)]
            for i in sorted(winner_index, reverse=True):
                self.winners.append((self.boards.pop(i), n))
            yield self.winners

    def draw_until_win(self):
        for winners in self.draw():
            if winners:
                return winners[0]
        return []

    def draw_until_last(self):
        for winners in self.draw():
            pass
        return winners[-1]


bingo_test = Bingo(bingo_text)
print(bingo_test.numbers)
winner, number = bingo_test.draw_until_win()
print(bingo_test.boards)

print("Winner: ", winner)
print(winner.sum_unmarked, number, number*winner.sum_unmarked)


[7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, 13, 6, 15, 25, 12, 22, 18, 20, 8, 19, 3, 26, 1]
[Board: [[(22, False), (13, False), (17, True), (11, True), (0, True)], [(8, False), (2, True), (23, True), (4, True), (24, True)], [(21, True), (9, True), (14, True), (16, False), (7, True)], [(6, False), (10, False), (3, False), (18, False), (5, True)], [(1, False), (12, False), (20, False), (15, False), (19, False)]], Board: [[(3, False), (15, False), (0, True), (2, True), (22, False)], [(9, True), (18, False), (13, False), (17, True), (5, True)], [(19, False), (8, False), (7, True), (25, False), (23, True)], [(20, False), (11, True), (10, False), (24, True), (4, True)], [(14, True), (21, True), (16, False), (12, False), (6, False)]]]
Winner:  Board: [[(14, True), (21, True), (17, True), (24, True), (4, True)], [(10, False), (16, False), (15, False), (9, True), (19, False)], [(18, False), (8, False), (23, True), (26, False), (20, False)], [(22, False), (11, True), (13, False), (6, Fals

In [99]:
bingo_test = Bingo(bingo_text)
loser, number = bingo_test.draw_until_last()

print("Loser: ", loser)
print(loser.sum_unmarked, number, number*loser.sum_unmarked)


Loser:  Board: [[(3, False), (15, False), (0, True), (2, True), (22, False)], [(9, True), (18, False), (13, True), (17, True), (5, True)], [(19, False), (8, False), (7, True), (25, False), (23, True)], [(20, False), (11, True), (10, True), (24, True), (4, True)], [(14, True), (21, True), (16, True), (12, False), (6, False)]]
148 13 1924


In [83]:
bingo = Bingo.from_file('4-puzzle-input.txt')
winner, number = bingo.draw_until_win()

print("Winner: ", winner)
print(winner.sum_unmarked, number, number*winner.sum_unmarked)


## Day 4 part 2

In [100]:
bingo = Bingo.from_file('4-puzzle-input.txt')
winner, number = bingo.draw_until_last()

print("Loser: ", winner)
print(winner.sum_unmarked, number, number*winner.sum_unmarked)


Loser:  Board: [[(45, False), (97, True), (96, True), (23, True), (62, True)], [(79, True), (59, True), (60, True), (87, True), (64, False)], [(75, True), (2, False), (30, True), (47, True), (50, True)], [(85, True), (81, True), (56, True), (11, False), (38, True)], [(17, True), (26, True), (40, True), (7, True), (66, True)]]
122 40 4880
